combine papers in the same topic to a topic network

In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import os
from IPython.core.debugger import set_trace


In [8]:
thisp

{'id': 'Ep63386',
 'label': 'Stahl2015_observing',
 'author': 'Stahl',
 'year': '2015',
 'title': 'Observing the unexpected enhances infants’ learning and exploration'}

In [43]:
def isrep(newstr,oldstr): # to try:different ways
    if newstr==oldstr:
        return True
    else:
        return False

In [12]:
np.where([False,False])[0]

array([], dtype=int64)

In [44]:
outdir='../database/curiosity/devpsych/'

pnames = ['Bonawitz2011-the','Stahl2015_observing']
topicQ = {'nodes':{'label':[],'reference':[]},'edges':{'start':[],'end':[]}}
for pname in pnames:
    # Q only network
    [Qndf,Qedf] = pickle.load(open(outdir+pname+'_Qonly.p','rb'))
    # full network
    [nodedf,edgedf,allngrps,thisp] = pickle.load(open(outdir+pname+'_qnet.p','rb'))

    # add new Q nodes into topicQ pool
    # 1-check repetition: Question similarity decision! 
    for kq in range(len(Qndf)):
        if Qndf.iloc[kq]['speaker']!=thisp['id']: # only include questions asked from this paper
            continue
        qidx = np.where(list(map(lambda x: isrep(Qndf['label'].iloc[kq],x),list(topicQ['nodes']['label']))))[0]
        if len(qidx)==0: # not found in current topic Q
            topicQ['nodes']['label'].append(Qndf['label'].iloc[kq])
            topicQ['nodes']['reference'].append({'paper':thisp,'content':Qndf['content'].iloc[kq]})
            qidx = len(topicQ['nodes']['label'])-1
        else:
            qidx = qidx[0]
        
        # add edges
        Eidx = np.where(topicQ['edges']['start']==topicQ['nodes']['label'][qidx])[0]
        qid = Qndf.iloc[kq]['id']
        outedge = edgedf.loc[edgedf['start']==qid]
        outq =[Qndf['label'].iloc[list(Qndf['id']).index(eid)] for eid in outedge['end'] if eid[0]=='q']
        if len(outq)>0:
            for kout in range(len(outq)):
                outlb = outq[kout]
                if len(np.where(list(map(lambda x: isrep(outlb,x),topicQ['edges']['end'])))[0])==0: # edge not found
                    topicQ['edges']['start'].append(topicQ['nodes']['label'][qidx])
                    topicQ['edges']['end'].append(outlb)
                    # TODO: add edge label
    
    
    # TODO: add answers, as a feature of each question node. mark where the answer comes from
    [nodedf,edgedf,allngrps,thisp] = pickle.load(open(outdir+pname+'_qnet.p','rb'))


In [51]:
pickle.dump([pd.DataFrame(topicQ['nodes']),pd.DataFrame(topicQ['edges'])],open(outdir+'topicnet.p','wb'))